In [119]:
from lxml import etree
import lxml.html
import wikitextparser as wtp
import nltk
import re

In [6]:
tree = etree.parse('C:\\Users\\achang\\Downloads\\simplewiki-20171103-pages-meta-current.xml')

In [7]:
namespaces = {
    'mediawiki': 'http://www.mediawiki.org/xml/export-0.10/'
}

In [591]:
index = 2502
page = tree.xpath(f'mediawiki:page[{index}]', namespaces = namespaces)[0]
redirect = tree.xpath(f'mediawiki:page[{index}]/mediawiki:redirect', namespaces = namespaces)
ns = tree.xpath(f'mediawiki:page[{index}]/mediawiki:ns', namespaces = namespaces)
title = tree.xpath(f'mediawiki:page[{index}]/mediawiki:title', namespaces = namespaces)[0].text
text = tree.xpath(f'mediawiki:page[{index}]/mediawiki:revision/mediawiki:text', namespaces = namespaces)[0].text
title, ns[0].text, redirect

('Phoebe (moon)', '0', [])

In [592]:
print(text)

[[File:Phoebe cassini.jpg|thumb|A picture of the surface of Phoebe]] 

'''Phoebe''' is a [[Satellite (natural)|moon]] which goes around ([[orbit]]s) the [[planet]] called [[Saturn (planet)|Saturn]]. It takes eighteen months for Phoebe to go all the way around Saturn. It is half made out of [[mineral|rock]], and half made out of [[ice]]. The ground on Phoebe is black, which means that it is very dark when seen from [[Earth]]. Phoebe does not have any [[air]], and there is no [[water]] on its [[surface]].

There are many [[crater]]s on Phoebe. These are from [[asteroids]] and other things crashing into it, and leaving round marks on the surface.

Phoebe is slightly irregular, which means that it is not perfectly round like a ball. It has an average [[diameter]] of 213&nbsp;km. This means that the Earth's moon is 15 times bigger than Phoebe. The duration between two sunrises (rotation period) is only 9 hours and 16.5 minutes long. 


{{Moons of Saturn}}

[[Category:Saturn's moons]]


In [566]:
def parse(raw_text):
    keep_tags = {'i','p','u','b','tt'}
    wiki_text = wtp.parse(raw_text)
    links = []
    strs = []
    offset = 0
    
    for l in wiki_text.wikilinks:
        span = l.span
        discard = re.match(r'^[^:]+:', l.target) and not l.target.startswith('wikt:')
        target = l.target if not discard and not l.target.startswith('wikt:') else None
        text = l.text or l.target
        links.append((span, discard, target, text))
        
    for l in wiki_text.external_links:
        span = l.span
        discard = False
        target = None
        text = l.text or l.url
        links.append((span, discard, target, text))
        
    for t in wiki_text.templates:
        span = t.span
        discard = True
        target = None
        text = None
        links.append((span, discard, target, text))
    
    for t in wiki_text.tags():
        span = t.span
        target = None
        try:
            discard = t.name not in keep_tags
            text = t.contents
        except TypeError:
            discard = True
            text = None
        links.append((span, discard, target, text))
    
    for t in wiki_text.tables:
        span = t.span
        discard = True
        target = None
        text = None
        links.append((span, discard, target, text))
    
    for l in wiki_text.lists():
        span = l.span
        discard = True
        target = None
        text = None
        links.append((span, discard, target, text))
        
    for pf in wiki_text.parser_functions:
        span = pf.span
        discard = True
        target = None
        text = None
        links.append((span, discard, target, text))
        
    for c in wiki_text.comments:
        span = c.span
        discard = True
        target = None
        text = None
        links.append((span, discard, target, text))
        
    links.sort()
    
    for (span, discard, target, text) in links:
        if span[0] < offset:
            continue
        strs.append(wiki_text.string[offset:span[0]])
        if not discard:
            if target:
                strs.append(f'_{text}_')
            else:
                strs.append(text)
        offset = span[1]
    
    strs.append(wiki_text.string[offset:])
    
    text = ''.join(strs)
    
    text = re.sub(r"''+", '', text)
    text = re.sub(r"==+[^=\n]+==+", '', text)
    text = re.sub('&nbsp;', ' ', text)
    text = re.sub('\n\n+', '\n\n', text)
    text = text.strip()
    
    return text

In [593]:
print(parse(text))

Phoebe is a _moon_ which goes around (_orbit_s) the _planet_ called _Saturn_. It takes eighteen months for Phoebe to go all the way around Saturn. It is half made out of _rock_, and half made out of _ice_. The ground on Phoebe is black, which means that it is very dark when seen from _Earth_. Phoebe does not have any _air_, and there is no _water_ on its _surface_.

There are many _crater_s on Phoebe. These are from _asteroids_ and other things crashing into it, and leaving round marks on the surface.

Phoebe is slightly irregular, which means that it is not perfectly round like a ball. It has an average _diameter_ of 213 km. This means that the Earth's moon is 15 times bigger than Phoebe. The duration between two sunrises (rotation period) is only 9 hours and 16.5 minutes long.


In [512]:
wiki_text = wtp.parse(text)

In [513]:
wiki_text.comments

#  'comments',
#  'external_links',
#  'insert',
#  'lists',
#  'nesting_level',
#  'parameters',
#  'parser_functions',


[Comment('<!-- interwiki -->')]

In [135]:
for bad in text2.xpath("//ref"):
  bad.getparent().remove(bad)

In [139]:
text2.text_content()

'Cosmology is the branch of _astronomy_ that deals with the origin, structure, evolution and space-time relationships of the _universe_.http://www.merriam-webster.com/dictionary/cosmology NASA defines cosmology as "The study of the structure and changes in the present universe".[http://genesismission.jpl.nasa.gov/educate/scimodule/Cosmogony/CosmogonyPDF/CosCosmolTT.pdf NASA education] Another definition of cosmology is "the study of the universe, and humanity\'s place in it".\n\nModern cosmology is dominated by the _Big Bang_ theory, which brings together observational astronomy and _particle physics_.http://oxforddictionaries.com/definition/cosmology \n\nThough the word cosmology is recent (first used in 1730 in _Christian Wolff_\'s Cosmologia Generalis), the study of the universe has a long history.\n\nHistory.\nUntil recently, people thought that the _universe_ was the _Milky Way_. They thought this because they could only see the planets up to _Saturn_ and _star_s. With the inventi

In [378]:
set([1,2,3])

{1, 2, 3}

In [565]:
re.match(r'^[^:]+:', 'de:bar')

<_sre.SRE_Match object; span=(0, 3), match='de:'>